In [114]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import pycountry
import kaleido
plotly.__version__

'5.3.1'

### Objective: To explore the energy data and related energy ESG indicators for countries to assess their progress towards clean energy and distribution to meet the energy requirements. 

#### Dataset Description: Energy Indicators 
(Source: https://datacatalog.worldbank.org/search/dataset/0037651/Environment--Social-and-Governance-Data)

The dataset is filtered from the bulk ESG data with the following columns.

- **NY.ADJ.DRES.GN.ZS	Adjusted savings: natural resources depletion (% of GNI)**

Natural resource depletion is the sum of net forest depletion, energy depletion, and mineral depletion. Net forest depletion is unit resource rents times the excess of roundwood harvest over natural growth. Energy depletion is the ratio of the value of the stock of energy resources to the remaining reserve lifetime. It covers coal, crude oil, and natural gas. Mineral depletion is the ratio of the value of the stock of mineral resources to the remaining reserve lifetime. It covers tin, gold, lead, zinc, iron, copper, nickel, silver, bauxite, and phosphate.

- **EN.CLC.CDDY.XD	Cooling Degree Days (projected change in number of degree Celsius)**

A cooling degree day (CDD) is a measurement designed to quantify the demand for energy needed to cool buildings. It is the number of degrees that a day's average temperature is above 18°C.

- **EG.IMP.CONS.ZS	Energy imports, net (% of energy use)**

Net energy imports are estimated as energy use less production, both measured in oil equivalents. A negative value indicates that the country is a net exporter. Energy use refers to use of primary energy before transformation to other end-use fuels, which is equal to indigenous production plus imports and stock changes, minus exports and fuels supplied to ships and aircraft engaged in international transport.

- **EG.EGY.PRIM.PP.KD	Energy intensity level of primary energy (MJ/2011 PPP GDP)**

Energy intensity level of primary energy is the ratio between energy supply and gross domestic product measured at purchasing power parity. Energy intensity is an indication of how much energy is used to produce one unit of economic output. Lower ratio indicates that less energy is used to produce one unit of output.

- **EG.USE.PCAP.KG.OE	Energy use (kg of oil equivalent per capita)**

Energy use refers to use of primary energy before transformation to other end-use fuels, which is equal to indigenous production plus imports and stock changes, minus exports and fuels supplied to ships and aircraft engaged in international transport.

- **EG.USE.COMM.FO.ZS	Fossil fuel energy consumption (% of total)**

Fossil fuel comprises coal, oil, petroleum, and natural gas products.

- **EG.FEC.RNEW.ZS	Renewable energy consumption (% of total final energy consumption)**

Renewable energy consumption is the share of renewables energy in total final energy consumption.



In [2]:
# Read the dataset
energyDF = pd.read_csv('energyIndicators.csv')
display(energyDF.info())

# Change Year dtype to string
energyDF['Year'] = energyDF['Year'].astype('str')
display(energyDF.info())
        
# Relative percentage of missing data
missingFreq = energyDF.isna().sum()/(len(energyDF))*100
missingFreq.sort_values(ascending=False, inplace=True)
missing = missingFreq.to_frame(name='value')
display(missing)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14818 entries, 0 to 14817
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country_Name       14818 non-null  object 
 1   Country_Code       14818 non-null  object 
 2   Year               14818 non-null  int64  
 3   EG.EGY.PRIM.PP.KD  4826 non-null   float64
 4   EG.IMP.CONS.ZS     8015 non-null   float64
 5   EG.USE.PCAP.KG.OE  7743 non-null   float64
 6   EG.USE.COMM.FO.ZS  8000 non-null   float64
 7   EG.FEC.RNEW.ZS     6661 non-null   float64
 8   NY.ADJ.DRES.GN.ZS  9153 non-null   float64
 9   EN.CLC.CDDY.XD     193 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 1.1+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14818 entries, 0 to 14817
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country_Name       14818 non-null  object 
 1   Country_Code       14818 non-null  object 
 2   Year               14818 non-null  object 
 3   EG.EGY.PRIM.PP.KD  4826 non-null   float64
 4   EG.IMP.CONS.ZS     8015 non-null   float64
 5   EG.USE.PCAP.KG.OE  7743 non-null   float64
 6   EG.USE.COMM.FO.ZS  8000 non-null   float64
 7   EG.FEC.RNEW.ZS     6661 non-null   float64
 8   NY.ADJ.DRES.GN.ZS  9153 non-null   float64
 9   EN.CLC.CDDY.XD     193 non-null    float64
dtypes: float64(7), object(3)
memory usage: 1.1+ MB


None

,value
EN.CLC.CDDY.XD,98.697530
EG.EGY.PRIM.PP.KD,67.431502
EG.FEC.RNEW.ZS,55.047915
EG.USE.PCAP.KG.OE,47.745985
EG.USE.COMM.FO.ZS,46.011608
EG.IMP.CONS.ZS,45.910379
NY.ADJ.DRES.GN.ZS,38.230530
Country_Name,0.000000
Country_Code,0.000000
Year,0.000000


In [3]:
energyDF['Country_Name'].nunique()

239

In [4]:
# List of countries from pycountry
countryLst = list(pycountry.countries)
result = map(lambda x: x.alpha_3, countryLst)
country_codes = (list(result))
display(country_codes)

['ABW',
 'AFG',
 'AGO',
 'AIA',
 'ALA',
 'ALB',
 'AND',
 'ARE',
 'ARG',
 'ARM',
 'ASM',
 'ATA',
 'ATF',
 'ATG',
 'AUS',
 'AUT',
 'AZE',
 'BDI',
 'BEL',
 'BEN',
 'BES',
 'BFA',
 'BGD',
 'BGR',
 'BHR',
 'BHS',
 'BIH',
 'BLM',
 'BLR',
 'BLZ',
 'BMU',
 'BOL',
 'BRA',
 'BRB',
 'BRN',
 'BTN',
 'BVT',
 'BWA',
 'CAF',
 'CAN',
 'CCK',
 'CHE',
 'CHL',
 'CHN',
 'CIV',
 'CMR',
 'COD',
 'COG',
 'COK',
 'COL',
 'COM',
 'CPV',
 'CRI',
 'CUB',
 'CUW',
 'CXR',
 'CYM',
 'CYP',
 'CZE',
 'DEU',
 'DJI',
 'DMA',
 'DNK',
 'DOM',
 'DZA',
 'ECU',
 'EGY',
 'ERI',
 'ESH',
 'ESP',
 'EST',
 'ETH',
 'FIN',
 'FJI',
 'FLK',
 'FRA',
 'FRO',
 'FSM',
 'GAB',
 'GBR',
 'GEO',
 'GGY',
 'GHA',
 'GIB',
 'GIN',
 'GLP',
 'GMB',
 'GNB',
 'GNQ',
 'GRC',
 'GRD',
 'GRL',
 'GTM',
 'GUF',
 'GUM',
 'GUY',
 'HKG',
 'HMD',
 'HND',
 'HRV',
 'HTI',
 'HUN',
 'IDN',
 'IMN',
 'IND',
 'IOT',
 'IRL',
 'IRN',
 'IRQ',
 'ISL',
 'ISR',
 'ITA',
 'JAM',
 'JEY',
 'JOR',
 'JPN',
 'KAZ',
 'KEN',
 'KGZ',
 'KHM',
 'KIR',
 'KNA',
 'KOR',
 'KWT',
 'LAO',


In [5]:
# Keep data for countries listed in the country_codes list
energyDF2 = energyDF[energyDF['Country_Code'].isin(country_codes)]
display(energyDF2['Country_Name'].nunique())

removed = energyDF[~energyDF['Country_Code'].isin(country_codes)]
display(removed['Country_Name'].unique())

193

array(['Arab World', 'Caribbean small states',
       'Central Europe and the Baltics', 'Early-demographic dividend',
       'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD)',
       'East Asia & Pacific (excluding high income)', 'Euro area',
       'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD)',
       'Europe & Central Asia (excluding high income)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',
       'IDA total', 'Late-demographic dividend',
       'Latin America & Caribbean',
       'Latin America & Caribbean (IDA & IBRD)',
       'Latin America & Caribbean (excluding high income)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle income',
       'Middle East & North Africa',
       'Middle East & North Africa (IDA & IBRD)',
       'Middle East & 

In [6]:
energyDF2.columns

Index(['Country_Name', 'Country_Code', 'Year', 'EG.EGY.PRIM.PP.KD',
       'EG.IMP.CONS.ZS', 'EG.USE.PCAP.KG.OE', 'EG.USE.COMM.FO.ZS',
       'EG.FEC.RNEW.ZS', 'NY.ADJ.DRES.GN.ZS', 'EN.CLC.CDDY.XD'],
      dtype='object')

We remove data for the years before 1971 and after 2014 because many countries have not reported data for those years

In [85]:
EG_USE_IMP = energyDF2[['Country_Name', 'Country_Code', 'Year', 'EG.USE.PCAP.KG.OE', 'EG.IMP.CONS.ZS']]

# Remove Years where no data is available for any country
years_to_remove = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967',
       '1968', '1969', '1970', '2015', '2016', '2017', '2018', '2019', '2020', '2050']

EG_USE_IMP = EG_USE_IMP[~EG_USE_IMP['Year'].isin(years_to_remove)]
display(EG_USE_IMP['Year'].nunique())



44

In [9]:
# Find countries with no missing data (all 44 years of data available)
grouped = EG_USE_IMP.groupby('Country_Code').count()
to_keep = grouped[grouped['EG.USE.PCAP.KG.OE'] == 44].reset_index()
display(to_keep)

EG_USE_IMP_lst1 = EG_USE_IMP[EG_USE_IMP['Country_Code'].isin(to_keep['Country_Code'])]
display(EG_USE_IMP_lst1['Country_Code'].nunique())

,Country_Code,Country_Name,Year,EG.USE.PCAP.KG.OE,EG.IMP.CONS.ZS
0,AGO,44,44,44,44
1,ALB,44,44,44,44
2,ARE,44,44,44,44
3,ARG,44,44,44,44
4,AUS,44,44,44,44
...,...,...,...,...,...
98,TUR,44,44,44,44
99,TZA,44,44,44,44
100,URY,44,44,44,44
101,USA,44,44,44,44


103

#### Top 15 Countries with highest average energy usage per capita

In [16]:
AvgEGUsage = EG_USE_IMP.drop(columns = ['EG.IMP.CONS.ZS']).groupby(['Country_Name', 'Country_Code']).mean()
AvgEGUsageTop15 = AvgEGUsage.reset_index().sort_values(by='EG.USE.PCAP.KG.OE', ascending=False)[0:14]
display(AvgEGUsageTop15)



,Country_Name,Country_Code,EG.USE.PCAP.KG.OE
140,Qatar,QAT,15441.892937
12,Bahrain,BHR,10070.710597
75,Iceland,ISL,9576.673511
101,Luxembourg,LUX,8924.329306
91,Kuwait,KWT,8643.283955
182,United Arab Emirates,ARE,8638.522628
31,Canada,CAN,7726.898005
184,United States,USA,7680.566271
175,Trinidad and Tobago,TTO,7049.564023
24,Brunei Darussalam,BRN,6687.948033


In [153]:
AvgEGUsageImp = EG_USE_IMP.groupby(['Country_Name', 'Country_Code']).mean()
sd = EG_USE_IMP.groupby(['Country_Name', 'Country_Code']).std()
display(sd)
AvgEGUsageImpTop15 = AvgEGUsageImp.reset_index().sort_values(by='EG.USE.PCAP.KG.OE', ascending=False)[0:14]
AvgEGUsageImpTop15 = AvgEGUsageImpTop15.merge(sd, how='left', on = 'Country_Name', suffixes=[None, '_sd'])
display(AvgEGUsageImpTop15)

# Barplots (Code modified from https://stackoverflow.com/questions/65766960/plotly-python-how-to-make-a-gapped-y-axis)
df= AvgEGUsageImpTop15.drop(columns = ['Country_Code'])
df.set_index('Country_Name', inplace=True)

# colors and cut-offs
colors = px.colors.qualitative.Plotly
cut_interval = [-75, 150, 3000]

# subplot setup
fig = make_subplots(rows=3, cols=1, vertical_spacing = 0.02)
fig.update_layout(title = "Top 15 Countries with the Highest Average Energy Use Per Capita <br><sup>Data Aggregated Over the Years 1971-2014 </sup>",
                )
legendNames = ['Average Energy Use (kg of oil equivalent per capita)', 'Average Energy Imports, net (% of Energy Use)']
# Traces for [3, 1]
# marker_color=colors[i] ensures that categories follow the same color cycle
for i, col in enumerate(df[['EG.USE.PCAP.KG.OE', 'EG.IMP.CONS.ZS']].columns):
    fig.add_trace(go.Bar(x=df.index,
                    y=df[col],
                    name=legendNames[i],
                    marker_color=colors[i],
                    legendgroup = col
                    ), row=3, col=1)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.4
    ))

# Traces for [2, 1]
# Notice that showlegend = False.
# Since legendgroup = col the interactivity is
# taken care of in the previous for-loop.
for i, col in enumerate(df[['EG.USE.PCAP.KG.OE', 'EG.IMP.CONS.ZS']].columns):
    fig.add_trace(go.Bar(x=df.index,
                    y=df[col],
                    name=col,
                    marker_color=colors[i],
                    legendgroup = col,
                    showlegend = False,
                    ), row=2, col=1)

# Traces for [1, 1]
# Notice that showlegend = False.
# Since legendgroup = col the interactivity is
# taken care of in the previous for-loop.
for i, col in enumerate(df[['EG.USE.PCAP.KG.OE', 'EG.IMP.CONS.ZS']].columns):
    fig.add_trace(go.Bar(x=df.index,
                    y=df[col],
                    name=col,
                    marker_color=colors[i],
                    legendgroup = col,
                    showlegend = False,
                    ), row=1, col=1)
    
# Some aesthetical adjustments to layout
fig.update_yaxes(range=[cut_interval[2], max(df.max()*1.1)], row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_yaxes(range=[ cut_interval[0], cut_interval[1]], row=2, col=1)
fig.update_xaxes(visible=False, row=2, col=1)
fig.update_yaxes(range=[min(df.min()*1.1), cut_interval[0]*1.1], row=3, col=1)
fig.update_xaxes(tickangle = -45, row=3, col=1)
fig.show()

# Save image
fig.write_image("energyUsePerCapita_Top15_Barplot.png", engine='kaleido', scale=1, width=800, height=600) 

,,EG.USE.PCAP.KG.OE,EG.IMP.CONS.ZS
Country_Name,Country_Code,,
Afghanistan,AFG,NaN,NaN
Albania,ALB,194.325494,33.185240
Algeria,DZA,273.781052,251.325179
Andorra,AND,NaN,NaN
Angola,AGO,57.699718,219.130609
...,...,...,...
"Venezuela, RB",VEN,243.712287,210.364927
Vietnam,VNM,135.153564,21.342408
"Yemen, Rep.",YEM,75.290318,190.685740


,Country_Name,Country_Code,EG.USE.PCAP.KG.OE,EG.IMP.CONS.ZS,EG.USE.PCAP.KG.OE_sd,EG.IMP.CONS.ZS_sd
0,Qatar,QAT,15441.892937,-637.076391,3694.539700,578.653639
1,Bahrain,BHR,10070.710597,-195.343218,1544.866472,145.147861
2,Iceland,ISL,9576.673511,29.308820,4079.772225,12.022071
3,Luxembourg,LUX,8924.329306,98.521952,1493.216692,0.978580
4,Kuwait,KWT,8643.283955,-732.912486,2007.377500,593.310837
5,United Arab Emirates,ARE,8638.522628,-1133.901731,2791.653089,1684.622846
6,Canada,CAN,7726.898005,-35.375237,436.677357,18.778105
7,United States,USA,7680.566271,18.527360,372.046761,5.947926
8,Trinidad and Tobago,TTO,7049.564023,-157.489819,4411.539154,87.302655
9,Brunei Darussalam,BRN,6687.948033,-1138.925291,1843.387940,964.682399


#### Top 15 Countries with highest average energy imports per capita

In [11]:
AvgEGImport = EG_USE_IMP.drop(columns = ['EG.USE.PCAP.KG.OE']).groupby(['Country_Name', 'Country_Code']).mean()
AvgEGImportTop15 = AvgEGImport.reset_index().sort_values(by='EG.IMP.CONS.ZS', ascending=False)[0:14]
display(AvgEGImportTop15)

,Country_Name,Country_Code,EG.IMP.CONS.ZS
107,Malta,MLT,99.870138
152,Singapore,SGP,99.026234
101,Luxembourg,LUX,98.521952
44,Cyprus,CYP,98.208932
85,Jordan,JOR,97.102782
113,Moldova,MDA,96.247951
95,Lebanon,LBN,94.263078
15,Belarus,BLR,87.112083
83,Jamaica,JAM,86.697627
84,Japan,JPN,84.355189


### Visualizations

#### Visualization for the yearly variation in Energy Usage Per Capita data for the top 10  countries with the highest average energy usage per capita

In [191]:
df = EG_USE_IMP[EG_USE_IMP['Country_Name'].isin(AvgEGUsageTop15['Country_Name'])].drop(columns=['EG.IMP.CONS.ZS'])
# Impute missing data with the average for the country
display(df.isna().sum())
df = df.set_index(['Country_Name', 'Country_Code', 'Year'])['EG.USE.PCAP.KG.OE'].fillna(AvgEGUsageTop15.set_index(['Country_Name', 'Country_Code'])['EG.USE.PCAP.KG.OE']).reset_index()
display(df.isna().sum())
fig = px.line(df, x='Year', y='EG.USE.PCAP.KG.OE',
             color='Country_Name',
             color_discrete_sequence=px.colors.qualitative.Dark24,
              labels= {'EG.USE.PCAP.KG.OE': 'Energy Use (kg of oil equivalent per capita)'},
                         title='Top 15 Countries with the Highest Average Energy Use Per Capita')
fig.show()

fig.write_image("energyUsePerCapita_Top15.png", engine='kaleido') 

Country_Name         0
Country_Code         0
Year                 0
EG.USE.PCAP.KG.OE    3
dtype: int64

Country_Name         0
Country_Code         0
Year                 0
EG.USE.PCAP.KG.OE    0
dtype: int64

In [189]:
df = EG_USE_IMP[EG_USE_IMP['Country_Name'].isin(AvgEGUsageTop15['Country_Name'])].drop(columns=['EG.USE.PCAP.KG.OE'])
# Impute missing data with the average for the country
display(df.isna().sum())
df = df.set_index(['Country_Name', 'Country_Code', 'Year'])['EG.IMP.CONS.ZS'].fillna(AvgEGImportTop15.set_index(['Country_Name', 'Country_Code'])['EG.IMP.CONS.ZS']).reset_index()

display(df.isna().sum())
fig = px.line(df, x='Year', y='EG.IMP.CONS.ZS',
             color='Country_Name',
             color_discrete_sequence=px.colors.qualitative.Dark24,
             labels= {'EG.IMP.CONS.ZS': 'Energy Imports, net (% of Energy Use)'},
             title='Top 15 Countries with the Highest Average Energy Use Per Capita')
fig.write_image("energyUsePerCapita_Top15EnergyUsers.png", engine='kaleido') 
fig.show()

Country_Name      0
Country_Code      0
Year              0
EG.IMP.CONS.ZS    0
dtype: int64

Country_Name      0
Country_Code      0
Year              0
EG.IMP.CONS.ZS    0
dtype: int64

#### Visualization for the yearly variation in Energy Import data for the top 10  countries with the highest average energy import per capita

In [190]:
df = EG_USE_IMP[EG_USE_IMP['Country_Name'].isin(AvgEGImportTop15['Country_Name'])].drop(columns=['EG.USE.PCAP.KG.OE'])
# Impute missing data with the average for the country
display(df.isna().sum())
df = df.set_index(['Country_Name', 'Country_Code', 'Year'])['EG.IMP.CONS.ZS'].fillna(AvgEGImportTop15.set_index(['Country_Name', 'Country_Code'])['EG.IMP.CONS.ZS']).reset_index()

display(df.isna().sum())
fig = px.line(df, x='Year', y='EG.IMP.CONS.ZS',
             color='Country_Name',
             color_discrete_sequence=px.colors.qualitative.Dark24,
             labels= {'EG.IMP.CONS.ZS': 'Energy Imports, net (% of Energy Use)'},
             title='Top 15 Countries with the Highest Average Energy Import Per Capita')
fig.show()
fig.write_image("energyUseImportCapita_Top15EnergyImporters.png", engine='kaleido') 

Country_Name       0
Country_Code       0
Year               0
EG.IMP.CONS.ZS    38
dtype: int64

Country_Name      0
Country_Code      0
Year              0
EG.IMP.CONS.ZS    0
dtype: int64

#### Conclusion: 

1. There has been a rise in per capita energy use over the years in most countries in top 15 list of energy users. This rise is also associated with a corresponding increase in the net energy import in these countries. 

2. Countries that were solely dependent on imports for their energy demands until late 1980's, such as Singapore, Malta, Jordan, and Luxembourg, are showing a decrease in energy imports in recent years. This could be attributed to an increase in the energy production in these countries potentially from renewable resources. Further analysis required. 

## Energy Consumption by Fuel Type

Here, 

1. we analyze the shares of renewable and fossil fuels in the total energy consumption for each country.
2. We give a 'green rank' to countries based on the ratio of energy consumption from renewables to fossil fuels.
3. Correlate the energy consumption to cooling degree days.

In [123]:
FuelTypeDF = energyDF2[['Country_Name', 'Country_Code', 'Year', 'EG.USE.PCAP.KG.OE', 'EG.USE.COMM.FO.ZS',
       'EG.FEC.RNEW.ZS', 'EN.CLC.CDDY.XD']]
FuelTypeDF = FuelTypeDF.rename(columns= {'EG.USE.PCAP.KG.OE': 'Energy_Use',
                                        'EG.USE.COMM.FO.ZS': 'Energy_Use_FF',
                                        'EG.FEC.RNEW.ZS': 'Energy_Use_RNEW',
                                        'EN.CLC.CDDY.XD': 'Cooling_Deg_Days'})
FuelTypeDF['Year'] = pd.to_datetime(FuelTypeDF['Year'])
display(FuelTypeDF.info())

# Remove Years where no data is available for any country
years_to_remove = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967',
       '1968', '1969', '1970', '2015', '2016', '2017', '2018', '2019', '2020', '2050']

FuelTypeDF = FuelTypeDF[~FuelTypeDF['Year'].isin(years_to_remove)].sort_values(by='Energy_Use', ascending=False)
display(FuelTypeDF)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11966 entries, 0 to 14817
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Country_Name      11966 non-null  object        
 1   Country_Code      11966 non-null  object        
 2   Year              11966 non-null  datetime64[ns]
 3   Energy_Use        5923 non-null   float64       
 4   Energy_Use_FF     5773 non-null   float64       
 5   Energy_Use_RNEW   5465 non-null   float64       
 6   Cooling_Deg_Days  193 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 747.9+ KB


None

,Country_Name,Country_Code,Year,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days
11080,Qatar,QAT,2004-01-01,22120.430302,100.000000,0.000000,NaN
11078,Qatar,QAT,2002-01-01,21300.073962,NaN,0.000000,NaN
11079,Qatar,QAT,2003-01-01,20727.866751,NaN,0.000000,NaN
11075,Qatar,QAT,1999-01-01,19839.010949,99.999991,0.000000,NaN
11077,Qatar,QAT,2001-01-01,19621.840514,99.999992,0.000000,NaN
...,...,...,...,...,...,...,...
14500,"Venezuela, RB",VEN,2014-01-01,NaN,NaN,12.004200,NaN
14562,Vietnam,VNM,2014-01-01,NaN,NaN,36.944401,NaN
14686,"Yemen, Rep.",YEM,2014-01-01,NaN,NaN,0.955600,NaN
14748,Zambia,ZMB,2014-01-01,NaN,NaN,86.002701,NaN


In [144]:
FuelTypeDF['Total'] = FuelTypeDF['Energy_Use_FF'] + FuelTypeDF['Energy_Use_RNEW']
display(FuelTypeDF.sort_values(by='Total', ascending=False))
display(FuelTypeDF.describe())

## Proportions of energy use from renewable vs fossil fuel sources don't always add up to a 100%

,Country_Name,Country_Code,Year,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days,Total
10092,Norway,NOR,2008-01-01,6752.708982,61.670897,58.489201,NaN,120.160097
901,Bangladesh,BGD,1993-01-01,126.798910,49.456842,70.557123,NaN,120.013965
10093,Norway,NOR,2009-01-01,6484.118585,63.110111,56.754200,NaN,119.864311
10094,Norway,NOR,2010-01-01,6934.590199,63.043064,56.378201,NaN,119.421265
10097,Norway,NOR,2013-01-01,6415.990714,61.840119,57.440601,NaN,119.280720
...,...,...,...,...,...,...,...,...
14500,"Venezuela, RB",VEN,2014-01-01,NaN,NaN,12.004200,NaN,NaN
14562,Vietnam,VNM,2014-01-01,NaN,NaN,36.944401,NaN,NaN
14686,"Yemen, Rep.",YEM,2014-01-01,NaN,NaN,0.955600,NaN,NaN
14748,Zambia,ZMB,2014-01-01,NaN,NaN,86.002701,NaN,NaN


,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days,Total
count,5608.000000,5470.000000,4701.000000,0.0,3440.000000
mean,2204.508728,65.024139,33.737467,NaN,94.899354
std,2640.165467,30.992760,31.300034,NaN,17.105838
min,9.548060,0.000000,0.000000,NaN,0.000000
25%,498.427885,39.615862,5.143344,NaN,94.226934
50%,1083.657731,74.893699,24.036877,NaN,99.825876
75%,2960.150289,92.002062,58.597531,NaN,102.388984
max,22120.430302,100.000000,98.342903,NaN,120.160097


In [155]:
gp = FuelTypeDF.groupby('Country_Code').count().sort_values(by='Energy_Use', ascending=False)

# Average aggregated over the years 1971-2014
Avg = FuelTypeDF.groupby(['Country_Name', 'Country_Code']).mean().sort_values(by='Energy_Use_RNEW', ascending=False)
sd = FuelTypeDF.groupby(['Country_Name', 'Country_Code']).std().sort_values(by='Energy_Use_RNEW', ascending=False)
Avg = Avg[Avg['Energy_Use_RNEW'].notna()].reset_index()
Avg['Green_Rank'] = Avg.index + 1
Avg = Avg.merge(sd, how='left', on='Country_Code', suffixes=[None, '_sd'])
display(Avg)

# Average between Years 1971-2000
yr_range_1 = (FuelTypeDF['Year'] >= '1971-01-01') & (FuelTypeDF['Year'] <= '2000-01-01')
Avg_1 = FuelTypeDF[yr_range_1].groupby(['Country_Name', 'Country_Code']).mean().sort_values(by='Energy_Use_RNEW', ascending=False)
Avg_1 = Avg_1[Avg_1['Energy_Use_RNEW'].notna()]
display(Avg_1)


# Average between Years 2001-2014
yr_range_2 = (FuelTypeDF['Year'] >= '2001-01-01') & (FuelTypeDF['Year'] <= '2014-01-01')
Avg_2 = FuelTypeDF[yr_range_2].groupby(['Country_Name', 'Country_Code']).mean().sort_values(by='Energy_Use_RNEW', ascending=False)
Avg_2 = Avg_2[Avg_2['Energy_Use_RNEW'].notna()]
display(Avg_2)

## There is not much difference in the rankings in the two time periods.



,Country_Name,Country_Code,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days,Total,Green_Rank,Energy_Use_sd,Energy_Use_FF_sd,Energy_Use_RNEW_sd,Cooling_Deg_Days_sd,Total_sd
0,"Congo, Dem. Rep.",COD,322.581246,7.328961,96.485776,NaN,100.716092,1,21.183629,4.106999,1.574472,NaN,1.681740
1,Ethiopia,ETH,479.049078,3.599869,95.044398,NaN,99.107936,2,5.738308,0.922344,1.589799,NaN,0.682511
2,Uganda,UGA,NaN,NaN,94.569601,NaN,NaN,3,NaN,NaN,1.322800,NaN,NaN
3,Burundi,BDI,NaN,NaN,94.342457,NaN,NaN,4,NaN,NaN,1.529989,NaN,NaN
4,Somalia,SOM,NaN,NaN,92.145529,NaN,NaN,5,NaN,NaN,2.062299,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Saudi Arabia,SAU,4136.646584,99.949357,0.013209,NaN,99.935709,187,1681.164029,0.149156,0.009486,NaN,0.197165
187,Palau,PLW,4877.028215,0.000000,0.000696,NaN,NaN,188,NaN,NaN,0.003336,NaN,NaN
188,Oman,OMN,2763.017297,99.909752,0.000040,NaN,99.839670,189,2104.533008,0.367507,0.000138,NaN,0.484148
189,Bahrain,BHR,10070.710597,99.774828,0.000000,NaN,99.530904,190,1544.866472,0.511989,0.000000,NaN,0.668658


,,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days,Total
Country_Name,Country_Code,,,,,
Chad,TCD,NaN,NaN,97.158379,NaN,NaN
Ethiopia,ETH,475.603998,3.064440,96.564258,NaN,99.757661
"Congo, Dem. Rep.",COD,322.965287,9.396978,96.226118,NaN,102.152750
Uganda,UGA,NaN,NaN,95.526129,NaN,NaN
Burundi,BDI,NaN,NaN,94.749193,NaN,NaN
...,...,...,...,...,...,...
Montenegro,MNE,NaN,NaN,0.000000,NaN,NaN
Timor-Leste,TLS,NaN,NaN,0.000000,NaN,NaN
Oman,OMN,1585.541485,99.999949,0.000000,NaN,99.999989


,,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days,Total
Country_Name,Country_Code,,,,,
"Congo, Dem. Rep.",COD,321.758300,2.897496,96.689793,NaN,99.587289
Burundi,BDI,NaN,NaN,94.022879,NaN,NaN
Ethiopia,ETH,486.431392,4.747216,93.850222,NaN,98.597438
Uganda,UGA,NaN,NaN,93.818043,NaN,NaN
Somalia,SOM,NaN,NaN,93.469564,NaN,NaN
...,...,...,...,...,...,...
Palau,PLW,NaN,NaN,0.001143,NaN,NaN
Kuwait,KWT,10415.571750,97.852942,0.000329,NaN,97.853402
Oman,OMN,5286.179752,99.737557,0.000071,NaN,99.737648


In [157]:
# Top 25 Green Ranks 
green_Top25 = Avg.sort_values(by='Green_Rank')[0:24]
display(green_Top25)


# Top 20 Energy Users 
energyUse_Top20 = Avg.sort_values(by='Energy_Use', ascending=False)[0:19]
display(energyUse_Top20)

## Top 20 average energy consumers have poor green ranking


,Country_Name,Country_Code,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days,Total,Green_Rank,Energy_Use_sd,Energy_Use_FF_sd,Energy_Use_RNEW_sd,Cooling_Deg_Days_sd,Total_sd
0,"Congo, Dem. Rep.",COD,322.581246,7.328961,96.485776,NaN,100.716092,1,21.183629,4.106999,1.574472,NaN,1.681740
1,Ethiopia,ETH,479.049078,3.599869,95.044398,NaN,99.107936,2,5.738308,0.922344,1.589799,NaN,0.682511
2,Uganda,UGA,NaN,NaN,94.569601,NaN,NaN,3,NaN,NaN,1.322800,NaN,NaN
3,Burundi,BDI,NaN,NaN,94.342457,NaN,NaN,4,NaN,NaN,1.529989,NaN,NaN
4,Somalia,SOM,NaN,NaN,92.145529,NaN,NaN,5,NaN,NaN,2.062299,NaN,NaN
5,Bhutan,BTN,277.925733,0.000000,92.010238,NaN,93.003530,6,101.093489,0.000000,2.652047,NaN,1.761327
6,Tanzania,TZA,432.363000,8.675362,91.221632,NaN,100.113443,7,47.949898,2.220142,3.410659,NaN,1.052057
7,Mozambique,MOZ,489.044185,8.005682,90.802197,NaN,97.607114,8,100.018262,2.311523,3.714195,NaN,2.477294
8,Chad,TCD,NaN,NaN,90.031626,NaN,NaN,9,NaN,NaN,6.886575,NaN,NaN
9,Nepal,NPL,326.278235,7.160610,89.926958,NaN,100.126070,10,34.007414,4.122221,2.871222,NaN,0.232714


,Country_Name,Country_Code,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days,Total,Green_Rank,Energy_Use_sd,Energy_Use_FF_sd,Energy_Use_RNEW_sd,Cooling_Deg_Days_sd,Total_sd
190,Qatar,QAT,15441.892937,99.999786,0.000000,NaN,99.999684,191,3694.539700,0.001014,0.000000,NaN,0.001242
189,Bahrain,BHR,10070.710597,99.774828,0.000000,NaN,99.530904,190,1544.866472,0.511989,0.000000,NaN,0.668658
43,Iceland,ISL,9576.673511,29.307254,63.350993,NaN,85.047067,44,4079.772225,12.024572,8.631061,NaN,1.473598
154,Luxembourg,LUX,8924.329306,89.240654,3.516024,NaN,90.728829,155,1493.216692,3.617031,1.772598,NaN,1.307668
183,Kuwait,KWT,8643.283955,99.387805,0.039103,NaN,98.895617,184,2007.377500,1.733045,0.083198,NaN,2.379600
179,United Arab Emirates,ARE,8638.522628,98.956445,0.094459,NaN,98.294970,180,2791.653089,3.115434,0.040304,NaN,3.949404
101,Canada,CAN,7726.898005,76.622893,21.865637,NaN,96.299874,102,436.677357,3.677471,0.323795,NaN,1.275848
147,United States,USA,7680.566271,87.818578,5.895700,NaN,91.241169,148,372.046761,3.513642,1.619501,NaN,0.727531
175,Trinidad and Tobago,TTO,7049.564023,99.307566,0.758627,NaN,100.342714,176,4411.539154,0.451497,0.345336,NaN,0.092677
182,Brunei Darussalam,BRN,6687.948033,98.978360,0.054712,NaN,100.073433,183,1843.387940,2.145933,0.158952,NaN,0.161730


### Visualizations

#### Bar plot for the Top 20 average energy consumers with ranking

In [192]:
df = energyUse_Top20
df['RENEW_Prop'] = df['Energy_Use'] * df['Energy_Use_RNEW']/100
fig = px.bar(df.sort_values(by='Energy_Use'), x=['Energy_Use'], y='Country_Name', text='Green_Rank', orientation='h',
            title="Top 20 Countries with the Highest Average Energy Use Per Capita with Green Ranking <br><sup>Data Aggregated Over the Years 1971-2014 </sup>",
            color='Energy_Use_RNEW', color_continuous_scale='greens', range_color=[0, 100],
             labels={'Country_Name': 'Country',
                    'Energy_Use_RNEW': 'Renewable Energy Proportion',
                    'value': 'Energy Use (kg of oil equivalent per capita)'}
            )

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

# Save image
fig.write_image("energyUsePerCapita_Top20_GreenRanking.png", engine='kaleido', scale=1, width=800, height=600) 

In [226]:
df = energyUse_Top20
df['RENEW_Prop'] = df['Energy_Use'] * (df['Energy_Use_RNEW']/100)
df['FF_Prop'] = df['Energy_Use'] - df['RENEW_Prop']
df = df.sort_values(by='Energy_Use')
display(df)
fig = px.bar(df, x=['FF_Prop', 'RENEW_Prop'], y='Country_Name', orientation='h',
            title="Top 20 Countries with the Highest Average Energy Use Per Capita with Green Ranking <br><sup>Data Aggregated Over the Years 1971-2014 </sup>",
            color_discrete_sequence=['red', 'green'],
             
            labels={'Country_Name': 'Country',
                    'value': 'Energy Use (kg of oil equivalent per capita)',
                   'variable': 'Energy Source'}
            )
newnames = {'FF_Prop': 'Fossil Fuels', 'RENEW_Prop': 'Renewables'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))
total_labels = [{"y": y, "x": total*1.05, "text": str(rank), "showarrow": False} for y, total, rank in zip(df['Country_Name'], df['Energy_Use'], df['Green_Rank'])]
fig = fig.update_layout(annotations=total_labels)
fig.show()

# Save image
fig.write_image("energyUsePerCapita_Top20_EnergyShareBreakdown_WithGreenRank.png", engine='kaleido', scale=1, width=800, height=600) 

,Country_Name,Country_Code,Energy_Use,Energy_Use_FF,Energy_Use_RNEW,Cooling_Deg_Days,Total,Green_Rank,Energy_Use_sd,Energy_Use_FF_sd,Energy_Use_RNEW_sd,Cooling_Deg_Days_sd,Total_sd,RENEW_Prop,FF_Prop
136,Czech Republic,CZE,4378.498453,90.549967,7.795413,NaN,93.033826,137,303.875868,7.694051,3.050039,NaN,2.258510,341.322050,4037.176402
158,Netherlands,NLD,4582.686769,95.851051,2.586528,NaN,96.822976,159,316.464551,2.509939,1.450301,NaN,1.048975,118.532459,4464.154310
153,Russian Federation,RUS,4699.699520,91.430125,3.596720,NaN,95.026845,154,517.824790,1.015863,0.230600,NaN,1.157174,169.035022,4530.664498
187,Palau,PLW,4877.028215,0.000000,0.000696,NaN,NaN,188,NaN,NaN,0.003336,NaN,NaN,0.033927,4876.994288
156,Belgium,BEL,4978.909311,79.660758,3.010412,NaN,77.681004,157,465.557070,8.255295,2.527402,NaN,1.193720,149.885692,4829.023618
134,Australia,AUS,5080.329509,93.482405,8.041881,NaN,102.145256,135,541.665264,1.358575,0.823707,NaN,1.500673,408.554058,4671.775451
46,Norway,NOR,5124.118566,56.204128,58.585260,NaN,114.783323,47,892.221388,3.127539,1.576039,NaN,2.517087,3001.978166,2122.140400
69,Sweden,SWE,5336.139330,44.916760,39.003555,NaN,73.580885,70,399.530272,15.321443,5.938351,NaN,4.084587,2081.284021,3254.855309
87,Finland,FIN,5697.088106,57.663788,30.710823,NaN,81.573710,88,894.162952,10.294769,4.550608,NaN,1.237910,1749.622664,3947.465442
182,Brunei Darussalam,BRN,6687.948033,98.978360,0.054712,NaN,100.073433,183,1843.387940,2.145933,0.158952,NaN,0.161730,3.659081,6684.288952
